In [185]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

ContinentDict = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_one():
    # Завантаження даних Energy Indicators.xls
    energy = pd.read_excel('C://Users//nazar//EnergyIndicators.xls', 
                          skiprows=17, skipfooter=38, usecols=[2, 3, 4, 5])
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # Заміна значень "..." на NaN
    energy = energy.replace('...', np.nan)

    # Перейменування країн
    country_rename_dict = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    energy['Country'] = energy['Country'].replace(country_rename_dict, regex=True)

    # Видалення дужок та чисел з назв країн
    energy['Country'] = energy['Country'].str.replace(r'\s*\(.*\)', '', regex=True)
    energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)

    # Перетворення Energy Supply в гігаджоулі
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000

    # Завантаження даних world_bank.csv
    GDP = pd.read_csv('C://Users//nazar//API_NY.GDP.MKTP.CD_DS2_en_csv_v2_5871885.csv', skiprows=4)
    GDP = GDP.rename(columns={'Country Name': 'Country'})

    # Перейменування країн
    GDP_rename_dict = {
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }
    GDP['Country'] = GDP['Country'].replace(GDP_rename_dict)

    # Вибірка лише років з 2006 по 2015
    GDP = GDP[['Country'] + list(map(str, range(2006, 2016)))]

    # Завантаження даних ScimEn.xlsx
    ScimEn = pd.read_excel('C://Users//nazar//scimagojrcountryrank1996-2022.xlsx')

    # Злиття даних
    merged = pd.merge(ScimEn, energy, on='Country', how='inner')
    merged = pd.merge(merged, GDP, on='Country', how='inner')
    merged = merged.set_index('Country')

    # Вибір стовпців
    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
               'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
               '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

    result = merged[columns].sort_values(by='Rank').head(15)

    return result

def answer_two():
    Top15 = answer_one()
    return Top15.loc[:, '2006':'2015'].mean(axis=1).sort_values(ascending=False).map('${:,.2f}'.format)

def answer_three():
    Top15 = answer_one()
    return Top15.loc[Top15.loc[:, '2006':'2015'].mean(axis=1).sort_values(ascending=False).index[5], '2015'] - Top15.loc[Top15.loc[:, '2006':'2015'].mean(axis=1).sort_values(ascending=False).index[5], '2006']

def answer_four():
    Top15 = answer_one()
    return Top15.assign(Citation_Ratio=Top15['Self-citations'] / Top15['Citations']).sort_values(by='Citation_Ratio', ascending=False).iloc[0].name, Top15.assign(Citation_Ratio=Top15['Self-citations'] / Top15['Citations']).sort_values(by='Citation_Ratio', ascending=False).iloc[0]['Citation_Ratio']

def answer_five():
    Top15 = answer_one()
    return Top15.assign(Population_Estimate=Top15['Energy Supply'] / Top15['Energy Supply per Capita']).sort_values(by='Population_Estimate', ascending=False).index[2]

def answer_six():
    Top15 = answer_one()
    return Top15.assign(Citable_Documents_Per_Person=Top15['Citable documents'] / (Top15['Energy Supply'] / Top15['Energy Supply per Capita'])).corr()['Citable_Documents_Per_Person']['Energy Supply per Capita']

def answer_seven():
    Top15 = answer_one()
    return Top15.assign(Population_Estimate=Top15['Energy Supply'] / Top15['Energy Supply per Capita']).groupby(ContinentDict)['Population_Estimate'].agg(['size', 'sum', 'mean', 'std']).reset_index().rename(columns={'Country': 'Continent'})


In [187]:
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,360468,358777,3947871,2705774,10.95,308,1.271910e+11,93.0,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551545e+12,8.532186e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,199442,195042,3068926,881789,15.39,422,9.083800e+10,286.0,11.570980,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,76103,74167,760964,280893,10.00,217,3.319500e+10,26.0,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856721e+12,2.039126e+12,2.103588e+12
Japan,4,56249,55680,633294,136132,11.26,217,1.898400e+10,149.0,10.232820,4.601663e+12,4.579750e+12,5.106679e+12,5.289494e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,52572,51156,909276,151672,17.30,267,7.920000e+09,124.0,10.600470,2.709978e+12,3.092996e+12,2.931684e+12,2.417566e+12,2.491397e+12,2.666403e+12,2.706341e+12,2.786315e+12,3.065223e+12,2.934858e+12
Germany,6,47781,46767,641717,133693,13.43,230,1.326100e+10,165.0,17.901530,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,43567,43290,175721,79765,4.03,103,3.070900e+10,214.0,17.288680,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,8,39036,38276,787010,125333,20.16,263,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,35991,34424,529459,123042,14.71,192,6.530000e+09,109.0,33.667230,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


In [188]:
answer_two()

Country
United States         $15,704,029,340,300.00
China                  $6,927,706,520,148.28
Japan                  $5,239,642,077,099.60
Germany                $3,523,341,675,097.46
United Kingdom         $2,780,276,197,282.68
France                 $2,691,336,579,143.64
Italy                  $2,142,986,168,498.18
Brazil                 $1,988,889,078,193.37
Russian Federation     $1,666,745,628,516.50
Canada                 $1,616,359,264,592.76
India                  $1,602,352,072,666.71
Spain                  $1,400,886,445,421.01
South Korea            $1,221,371,625,499.97
Australia              $1,207,512,888,804.88
Iran                     $456,326,122,839.82
dtype: object

In [189]:
answer_three()

118652421857.7998

In [190]:
answer_four()

('China', 0.6853754846599598)

In [191]:
answer_five()

'United States'

In [192]:
answer_six()

0.7114342527331206

In [193]:
answer_seven()

,Continent,size,sum,mean,std
0,Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
1,Australia,1,2.331602e+07,2.331602e+07,NaN
2,Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
3,North America,2,3.528552e+08,1.764276e+08,1.996696e+08
4,South America,1,2.059153e+08,2.059153e+08,NaN
